In [1]:
import geopandas
import geopandas as gpd
import pandas as pd
from osgeo import ogr,gdal
import numpy 
from shapely.wkb import loads
import pygeos
import matplotlib.pyplot as plt
import openpyxl
import itertools
import networkx as nx
from simplify import *

import warnings
warnings.filterwarnings("ignore")

In [53]:
gdal.SetConfigOption("OSM_CONFIG_FILE", "osmconf.ini")

# Define a helper function to generate pairs of consecutive elements in a list
def pairwise(iterable):
    "s -> (s0, s1), (s1, s2), (s2, s3), ..."
    a, b = itertools.tee(iterable)
    next(b, None)
    return zip(a, b)

def permutations(iterable, r=None):
   # permutations('ABCD', 2) --> AB AC AD BA BC BD CA CB CD DA DB DC
    # permutations(range(3)) --> 012 021 102 120 201 210
    pool = tuple(iterable)
    n = len(pool)
    r = n if r is None else r
    if r > n:
        return
    indices = list(range(n))
    cycles = list(range(n, n-r, -1))
    yield tuple(pool[i] for i in indices[:r])
    while n:
        for i in reversed(range(r)):
            cycles[i] -= 1
            if cycles[i] == 0:
                indices[i:] = indices[i+1:] + indices[i:i+1]
                cycles[i] = n - i
            else:
                j = cycles[i]
                indices[i], indices[-j] = indices[-j], indices[i]
                yield tuple(pool[i] for i in indices[:r])
                break
        else:
            return 
        
        
# Retrive data from OSM and get the geographic data of subway
def query_b(geoType,keyCol,**valConstraint):
    """
    This function builds an SQL query from the values passed to the retrieve() function.
    Arguments:
         *geoType* : Type of geometry (osm layer) to search for.
         *keyCol* : A list of keys/columns that should be selected from the layer.
         ***valConstraint* : A dictionary of constraints for the values. e.g. WHERE 'value'>20 or 'value'='constraint'
    Returns:
        *string: : a SQL query string.
    """
    query = "SELECT " + "osm_id"
    for a in keyCol: query+= ","+ a  
    query += " FROM " + geoType + " WHERE "
    # If there are values in the dictionary, add constraint clauses
    if valConstraint: 
        for a in [*valConstraint]:
            # For each value of the key, add the constraint
            for b in valConstraint[a]: query += a + b
        query+= " AND "
    # Always ensures the first key/col provided is not Null.
    query+= ""+str(keyCol[0]) +" IS NOT NULL" 
    return query 

def retrieve(osm_path,geoType,keyCol,**valConstraint):
    """
    Function to extract specified geometry and keys/values from OpenStreetMap
    Arguments:
        *osm_path* : file path to the .osm.pbf file of the region 
        for which we want to do the analysis.     
        *geoType* : Type of Geometry to retrieve. e.g. lines, multipolygons, etc.
        *keyCol* : These keys will be returned as columns in the dataframe.
        ***valConstraint: A dictionary specifiying the value constraints.  
        A key can have multiple values (as a list) for more than one constraint for key/value.  
    Returns:
        *GeoDataFrame* : a geopandas GeoDataFrame with all columns, geometries, and constraints specified.    
    """
    driver=ogr.GetDriverByName('OSM')
    data = driver.Open(osm_path)
    query = query_b(geoType,keyCol,**valConstraint)
    sql_lyr = data.ExecuteSQL(query)
    features =[]
    # cl = columns 
    cl = ['osm_id']
    
    
    for a in keyCol: cl.append(a)
    if data is not None:
        for feature in sql_lyr:
            try:
                if feature.GetField(keyCol[0]) is not None:
                    pygeos_geo = pygeos.from_wkt(feature.geometry().ExportToWkt())
                    if pygeos_geo is None:
                        continue
                    # field will become a row in the dataframe.
                    field = []
                    for i in cl: field.append(feature.GetField(i))
                    field.append(pygeos_geo)   
                    features.append(field)
            except:
                print("WARNING: skipped OSM feature")
      
    cl.append('geometry')                   
    if len(features) > 0:
        return geopandas.GeoDataFrame(features,columns=cl) #,crs={'init': 'epsg:4326'}
    else:
        print("WARNING: No features or No Memory. returning empty GeoDataFrame") 
        return geopandas.GeoDataFrame(columns=['osm_id','geometry']) #,crs={'init': 'epsg:4326'}
    
def railway(osm_path):
    """
    Function to extract railway linestrings from OpenStreetMap
    Arguments:
        *osm_path* : file path to the .osm.pbf file of the region 
        for which we want to do the analysis.       
    Returns:
        *GeoDataFrame* : a geopandas GeoDataFrame with all unique land-use polygons.
    """ 
    return retrieve(osm_path,'lines',['railway','service'])

def subway_network(osm_path):
    """
    Extract subway network information from an OpenStreetMap XML file.

    Args:
    - osm_path: a string representing the path to the OpenStreetMap XML file

    Returns:
    - a pandas DataFrame containing information about subway railways in the OpenStreetMap data

    Example:
    >>> import pandas as pd
    >>> osm_path = 'path/to/osm/file.xml'
    >>> subway_network = subway_network(osm_path)
    >>> print(subway_network.head())
          osmid        name        railway  ...    ref tunnel        geometry
    1234  1234    Line 1    subway   ...  M1    yes  LINESTRING (0.0 0.0, 1.0 1.0)
    5678  5678    Line 2    subway   ...  M2    yes  LINESTRING (1.0 1.0, 2.0 2.0)

    Note: This function requires the osmnx library to be installed and OpenStreetMap data with railway information to be available.
    """
    # Use the osmnx library to extract railway information from the OpenStreetMap data
    df_railway = railway(osm_path)
    
    # Filter the DataFrame to include only subway railways
    subway = df_railway.loc[df_railway.railway == 'subway']
    
    return subway


# Pre-processing the geographic data of the subway network to obtain 'edges' and 'nodes'
def prepare_network(subway):
    """
    Prepare a subway network represented as a GeoDataFrame of LineString objects for routing.

    Args:
    - subway: a GeoDataFrame representing the subway network, with 'geometry' column containing LineString objects
    
    Returns:
    - a tuple of two DataFrames representing the edges and nodes of the prepared network

    Example:
    >>> import geopandas
    >>> subway = geopandas.read_file('subway.shp')
    >>> edges, nodes = prepare_network(subway)

    Note: This function requires the PyGEOS, NetworkX, and Pandas libraries to be installed.
    """
    # Convert the subway GeoDataFrame's 'geometry' column from Shapely objects to PyGEOS objects
    df_subway = pd.DataFrame(subway.copy())
    df_subway.geometry = pygeos.from_shapely(df_subway.geometry)
    
    # Build a Network object from the subway edges
    net = Network(edges=df_subway)

    # Add endpoints to the network where edges don't intersect
    net = add_endpoints(net)

    # Split edges at new endpoints
    net = split_edges_at_nodes(net)

    # Add new endpoints where edges were split
    net = add_endpoints(net)

    # Assign unique IDs to nodes and edges
    net = add_ids(net)

    # Add missing topology information to the network's edges
    net = add_topology(net)    

    # Calculate the degree of each node in the network
    net.nodes['degree'] = calculate_degree(net)

    # Merge edges with a degree of 2
    net = merge_edges(net)

    # Drop duplicate edges
    net.edges = drop_duplicate_geometries(net.edges, keep='first') 

    # Reset node and edge IDs after fixing topology and merging edges
    net = reset_ids(net) 

    # Add edge distances
    net = add_distances(net)

    # Merge any MultiLineString edges
    net = merge_multilinestrings(net)

    # Add travel time for each edge based on distance and average speed
    net = add_travel_time(net)
    
    # Return the edges and nodes of the prepared network
    return net.edges, net.nodes

def expand_edges(edges):
    """
    Expand a DataFrame of edges into a format that can be used with network analysis algorithms.

    Args:
    - edges: a DataFrame containing edges with columns 'from_id', 'to_id', and 'distance'

    Returns:
    - a DataFrame containing expanded edges with columns 'from_id', 'to_id', 'distance', 'weights', 'to_from', and 'from_to'

    Example:
    >>> edges = pd.DataFrame({'from_id': [0, 0, 1], 'to_id': [1, 2, 2], 'distance': [1.5, 3.2, 2.8]})
    >>> expand_edges(edges)
      from_id  to_id  distance  weights  to_from  from_to
    0       0      1       1.5        1  (0, 1)  (1, 0)
    1       0      2       3.2        3  (0, 2)  (2, 0)
    2       1      2       2.8        2  (1, 2)  (2, 1)

    Notes:
    - The 'weights' column is created by rounding the 'distance' column to the nearest integer.
    - The 'to_from' and 'from_to' columns are created to facilitate conversion between edge formats.
    """
    # Round the distance to the nearest integer and store it in a new column 'weights'
    edges['weights'] = edges['distance'].astype(int)

    # Create 'to_from' and 'from_to' columns to facilitate conversion between edge formats
    edges['to_from'] = list(zip(edges.from_id, edges.to_id))
    edges['from_to'] = list(zip(edges.to_id, edges.from_id))

    # Return the expanded edges DataFrame
    return edges


# Create a base Graph object as the basic topology network with 'edges' and 'nodes'
def create_ground_graph(edges, nodes):
    """
    Create a networkx Graph object representing a ground transportation network.

    Args:
    - edges: a pandas DataFrame containing edges in the network, with columns 'from_id', 'to_id', and 'weights'
    - nodes: a GeoDataFrame containing nodes in the network, with a 'geometry' column representing their coordinates as PyGEOS Point objects
    
    Returns:
    - a networkx Graph object representing the ground transportation network

    Example:
    >>> import pandas as pd
    >>> import geopandas as gpd
    >>> import networkx as nx
    >>> edges = pd.read_csv('edges.csv')
    >>> nodes = gpd.read_file('nodes.shp')
    >>> G = create_ground_graph(edges, nodes)

    Note: This function requires the pandas, geopandas, and networkx libraries to be installed.
    """
    # Extract the edges from the input DataFrame and create a list of tuples with weights
    od = edges[['from_id', 'to_id', 'weights']]
    edges_list = []
    for i, row in od.iterrows():
        weight_dict = {"weight": row[2]}
        tuple_row = (row[0], row[1], weight_dict)
        edges_list.append(tuple_row)
    
    # Extract the nodes from the input GeoDataFrame and create a list of node IDs
    nodes_list = nodes.iloc[:,2].tolist()
    
    # Create an empty Graph object and add the nodes and edges
    G = nx.Graph()
    G.add_nodes_from(nodes_list)
    G.add_edges_from(edges_list)
    
    return G


# Create coordinates of random start and end points in the research area
def create_grid(bbox,height):
    """Create a vector-based grid
    Args:
        bbox ([type]): [description]
        height ([type]): [grid size]
    Returns:
        [type]: [description]
    """    

    # set xmin,ymin,xmax,and ymax of the grid
    xmin, ymin = pygeos.total_bounds(bbox)[0],pygeos.total_bounds(bbox)[1]
    xmax, ymax = pygeos.total_bounds(bbox)[2],pygeos.total_bounds(bbox)[3]
    
    #estimate total rows and columns
    rows = int(np.ceil((ymax-ymin) / height))
    cols = int(np.ceil((xmax-xmin) / height))

    # set corner points
    x_left_origin = xmin
    x_right_origin = xmin + height
    y_top_origin = ymax
    y_bottom_origin = ymax - height

    # create actual grid
    res_geoms = []
    for countcols in range(cols):
        y_top = y_top_origin
        y_bottom = y_bottom_origin
        for countrows in range(rows):
            res_geoms.append((
                ((x_left_origin, y_top), (x_right_origin, y_top),
                (x_right_origin, y_bottom), (x_left_origin, y_bottom)
                )))
            y_top = y_top - height
            y_bottom = y_bottom - height
        x_left_origin = x_left_origin + height
        x_right_origin = x_right_origin + height

    return pygeos.polygons(res_geoms)

def coordinates_pairs(create_grid):
    grid_center_coordinates = gpd.GeoDataFrame(pygeos.centroid(grid.geometry.values),columns=['geometry'])
    coordinates_pairs = gpd.GeoDataFrame(itertools.permutations(grid_center_coordinates.geometry, 2), columns=['s_coordinates', 'e_coordinates'])
    return pd.DataFrame(coordinates_pairs.copy())


# Define nearest nodes with real_world coordinates of start and end points, obtain the nearest id pairs of nodes 
def find_nearest_node(coordinate, nodes):
    """
    Find the nearest node to a given coordinate or geometry in a GeoDataFrame of nodes.

    Args:
    - coordinate: a tuple of longitude and latitude (in decimal degrees) or a PyGEOS geometry object representing the location to search from
    - nodes: a GeoDataFrame containing nodes with a 'geometry' column representing their coordinates as PyGEOS Point objects
    
    Returns:
    - the id value of the nearest node to the input coordinate or geometry

    Example:
    >>> import geopandas
    >>> from shapely.geometry import Point
    >>> from pygeos import Geometry
    >>> nodes = geopandas.read_file('nodes.shp')
    >>> coordinate = (-122.3, 47.6)
    >>> find_nearest_node(coordinate, nodes)
    1234

    Note: This function requires the PyGEOS and STRtree libraries to be installed.
    """
    # Build an STRtree index of the nodes' geometries for efficient nearest-neighbor search
    node_tree = pygeos.STRtree(nodes.geometry)
   
    # Find the nearest node to the input coordinate or geometry using the STRtree index
    if isinstance(coordinate, tuple):
        find_nearest = node_tree.nearest(pygeos.points(coordinate))
    elif isinstance(coordinate, pygeos.lib.Geometry):
        find_nearest = node_tree.nearest(coordinate)
    
    # Return the id value of the nearest node from the nodes GeoDataFrame
    return int(nodes.iloc[find_nearest[1]]['id'])

def id_pairs(coordinates_pairs, nodes):
    pairs_num = coordinates_pairs.shape[0]
    id_pairs = pd.DataFrame(columns=['s_id','e_id'])
    for i in range(pairs_num):
        s_coordinate = pygeos.points(coordinates_pairs.s_coordinates[i])
        s_id = find_nearest_node(s_coordinate, nodes)
        e_coordinate = pygeos.points(coordinates_pairs.e_coordinates[i])
        e_id = find_nearest_node(e_coordinate, nodes)
        id_pairs.loc[i] = [s_id,e_id]
        
    return id_pairs


# Calculate the shorted path
def shortest_path(G, start_point_id, end_point_id, edges, weight = "weight"):
    """
    Compute the shortest path between two nodes in a given graph, along with its length and the edges that belong to the path.

    Args:
    - G: a networkx Graph object representing the graph
    - start_point_id: the ID of the node where the path starts
    - end_point_id: the ID of the node where the path ends
    - weight: the attribute used to determine the weight of the edges in the graph (default: "weight")

    Returns:
    - a tuple containing:
        - the list of nodes that form the shortest path from start_point_id to end_point_id
        - the length of the shortest path
        - a pandas DataFrame containing the edges that belong to the shortest path, with a new column 'linewidth' that is proportional to the edge weights

    Example:
    >>> import networkx as nx
    >>> import pandas as pd
    >>> G = nx.Graph()
    >>> G.add_edge(0, 1, weight=2.0)
    >>> G.add_edge(1, 2, weight=1.0)
    >>> G.add_edge(0, 2, weight=3.0)
    >>> path, length, edges = shortest_path(G, 0, 2)
    >>> print(path)
    [0, 1, 2]
    >>> print(length)
    3.0
    >>> print(edges)
       from_id  to_id  weights  to_from  from_to  linewidth
    0        0      1      2.0      NaN      1.0        1.0
    2        1      2      1.0      2.0      NaN        1.0
    """
    
    # Compute the shortest path and its length using the networkx library
    path_s_e = nx.shortest_path(G, source=start_point_id, target=end_point_id, weight= weight)
    length_s_e = nx.shortest_path_length(G, source=start_point_id, target=end_point_id, weight= weight)
    
   
    # Select the edges that belong to the shortest path and compute their linewidth based on their weight
    short_path_edges = edges.loc[edges.to_from.isin(list(pairwise(path_s_e))) | edges.from_to.isin(list(pairwise(path_s_e)))]
    short_path_edges['linewidth'] = short_path_edges['weights'].apply(lambda x: np.ceil(x * 0.01 / 2))
    
    # Return the computed values as a tuple
    return path_s_e, length_s_e, short_path_edges

In [3]:
osm_path = "C:\\projects\\UTNCE\\data\\Amsterdam.osm.pbf"

In [4]:
subway = subway_network(osm_path)

In [ ]:
# subway.plot()

In [6]:
edges,nodes = prepare_network(subway)

topology: 100%|████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 4196.97it/s]


In [7]:
edges = expand_edges(edges)
edges

,osm_id,geometry,railway,service,id,from_id,to_id,distance,time,weights,to_from,from_to
0,23157994,"LINESTRING (4.922 52.338, 4.922 52.338, 4.923 ...",subway,NaN,0,294,20,858.377718,0.042919,858,"(294, 20)","(20, 294)"
1,23189033,"LINESTRING (4.846 52.395, 4.847 52.395, 4.847 ...",subway,NaN,1,115,120,237.962864,0.011898,237,"(115, 120)","(120, 115)"
2,23189039,"LINESTRING (4.839 52.386, 4.839 52.386, 4.839 ...",subway,NaN,2,200,115,1320.099057,0.066005,1320,"(200, 115)","(115, 200)"
3,23189059,"LINESTRING (4.837 52.377, 4.837 52.378, 4.837 ...",subway,NaN,3,172,201,902.754616,0.045138,902,"(172, 201)","(201, 172)"
4,23189063,"LINESTRING (4.839 52.385, 4.84 52.385, 4.84 52...",subway,NaN,4,191,173,850.965721,0.042548,850,"(191, 173)","(173, 191)"
...,...,...,...,...,...,...,...,...,...,...,...,...
387,1061085945,"LINESTRING (4.922 52.339, 4.922 52.338, 4.922 ...",subway,NaN,387,198,294,34.811758,0.001741,34,"(198, 294)","(294, 198)"
388,1131538376,"LINESTRING (4.921 52.339, 4.921 52.34, 4.921 5...",subway,NaN,388,64,58,348.667904,0.017433,348,"(64, 58)","(58, 64)"
389,1131538382,"LINESTRING (4.922 52.339, 4.921 52.34, 4.921 5...",subway,NaN,389,97,98,350.467868,0.017523,350,"(97, 98)","(98, 97)"
390,1131538383,"LINESTRING (4.92 52.342, 4.92 52.342, 4.92 52....",subway,NaN,390,98,136,307.079253,0.015354,307,"(98, 136)","(136, 98)"


In [27]:
nodes

,geometry,degree,id
0,POINT (4.853 52.338),3,0
1,POINT (4.958 52.331),3,1
2,POINT (4.987 52.309),1,2
3,POINT (4.97 52.33),4,3
4,POINT (4.97 52.329),3,4
...,...,...,...
292,POINT (4.852 52.395),3,292
293,POINT (4.855 52.395),1,293
294,POINT (4.922 52.338),3,294
295,POINT (4.922 52.338),3,295


In [51]:
grid = pd.DataFrame(create_grid(edges.geometry,0.01),columns=['geometry'])

In [54]:
coordinates_pairs = coordinates_pairs(grid)
coordinates_pairs

,s_coordinates,e_coordinates
0,POINT (4.838860099999999 52.4009965),POINT (4.838860099999999 52.3909965)
1,POINT (4.838860099999999 52.4009965),POINT (4.838860099999999 52.38099650000001)
2,POINT (4.838860099999999 52.4009965),POINT (4.838860099999999 52.37099650000001)
3,POINT (4.838860099999999 52.4009965),POINT (4.838860099999999 52.36099650000001)
4,POINT (4.838860099999999 52.4009965),POINT (4.838860099999999 52.35099650000001)
...,...,...
41407,POINT (4.998860099999996 52.29099650000002),POINT (4.998860099999996 52.34099650000001)
41408,POINT (4.998860099999996 52.29099650000002),POINT (4.998860099999996 52.330996500000005)
41409,POINT (4.998860099999996 52.29099650000002),POINT (4.998860099999996 52.32099650000001)
41410,POINT (4.998860099999996 52.29099650000002),POINT (4.998860099999996 52.310996500000016)


In [ ]:
# coordinates_pairs.to_excel('./coordinates_pairs.xlsx',header = True, index = False) 

In [55]:
id_pairs = id_pairs(coordinates_pairs,nodes)
id_pairs
# id_pairs.drop_duplicates()

,s_id,e_id
0,115,192
1,115,173
2,115,183
3,115,180
4,115,168
...,...,...
41407,78,202
41408,78,124
41409,78,109
41410,78,109


In [ ]:
# id_pairs.to_excel('./id_pairs.xlsx',header = True, index = False) 

In [28]:
G = create_ground_graph(edges, nodes)
G

In [30]:
row_num = id_pairs.shape[0]
row_num

41412

In [45]:
for i in range(row_num):
    s_id = id_pairs.loc[i,'s_id']
    e_id = id_pairs.loc[i,'e_id']
    try:
        path_s_e = nx.shortest_path(G, source=s_id, target=e_id, weight= "weight")
        length_s_e = nx.shortest_path_length(G, source=s_id, target=e_id, weight= "weight")
        short_path_edges = edges.loc[edges.to_from.isin(list(pairwise(path_s_e))) | edges.from_to.isin(list(pairwise(path_s_e)))]
        short_path_edges['linewidth'] = short_path_edges['weights'].apply(lambda x: np.ceil(x * 0.01 / 2))
    except nx.NetworkXNoPath:
        print(f"No path found between {s_id} and {e_id}")
        continue
    print(path_s_e, length_s_e)

[115, 114, 192] 1414
[115, 200, 191, 173] 2229
[115, 200, 191, 173, 172, 184, 183] 2333
[115, 200, 191, 173, 172, 184, 205, 188, 180] 4192
[115, 200, 191, 173, 172, 184, 205, 181, 168] 5425
[115, 200, 191, 173, 172, 184, 205, 188, 180, 204, 169, 185, 186] 5533
[115, 200, 191, 173, 172, 184, 205, 188, 180, 204, 169, 185, 38] 7666
[115, 200, 191, 173, 172, 184, 205, 188, 180, 204, 169, 185, 38] 7666
[115, 200, 191, 173, 172, 184, 205, 188, 180, 204, 169, 185, 38] 7666
[115, 200, 191, 173, 172, 184, 205, 188, 180, 204, 169, 185, 38] 7666
[115, 200, 191, 173, 172, 184, 205, 188, 180, 204, 169, 185, 38] 7666
[115, 120, 261, 116, 290, 260, 289, 121] 636
[115, 114, 119, 253, 113] 586
[115, 200, 201] 1378
[115, 200, 191, 173, 172, 184] 2291
[115, 200, 191, 173, 172, 184, 205, 181] 4275
[115, 200, 191, 173, 172, 184, 205, 188, 180, 204, 169, 185, 186, 39] 7645
[115, 200, 191, 173, 172, 184, 205, 188, 180, 204, 169, 185, 186, 39] 7645
[115, 200, 191, 173, 172, 184, 205, 188, 180, 204, 169, 185, 

In [26]:
for i in range(row_num):
    s_id = id_pairs.loc[i,'s_id']
    e_id = id_pairs.loc[i,'e_id']
    path_s_e = nx.shortest_path(G, source=s_id, target=e_id, weight= "weight")
    
    if type(path_s_e)== "list":
        continue
    length_s_e = nx.shortest_path_length(G, source=s_id, target=e_id, weight= "weight")
    short_path_edges = edges.loc[edges.to_from.isin(list(pairwise(path_s_e))) | edges.from_to.isin(list(pairwise(path_s_e)))]
    short_path_edges['linewidth'] = short_path_edges['weights'].apply(lambda x: np.ceil(x * 0.01 / 2))
        
    print(path_s_e, length_s_e)

[115, 114, 192] 1414
[115, 200, 191, 173] 2229
[115, 200, 191, 173, 172, 184, 183] 2333
[115, 200, 191, 173, 172, 184, 205, 188, 180] 4192
[115, 200, 191, 173, 172, 184, 205, 181, 168] 5425
[115, 200, 191, 173, 172, 184, 205, 188, 180, 204, 169, 185, 186] 5533
[115, 200, 191, 173, 172, 184, 205, 188, 180, 204, 169, 185, 38] 7666
[115, 200, 191, 173, 172, 184, 205, 188, 180, 204, 169, 185, 38] 7666
[115, 200, 191, 173, 172, 184, 205, 188, 180, 204, 169, 185, 38] 7666
[115, 200, 191, 173, 172, 184, 205, 188, 180, 204, 169, 185, 38] 7666
[115, 200, 191, 173, 172, 184, 205, 188, 180, 204, 169, 185, 38] 7666
[115, 120, 261, 116, 290, 260, 289, 121] 636
[115, 114, 119, 253, 113] 586
[115, 200, 201] 1378
[115, 200, 191, 173, 172, 184] 2291
[115, 200, 191, 173, 172, 184, 205, 181] 4275
[115, 200, 191, 173, 172, 184, 205, 188, 180, 204, 169, 185, 186, 39] 7645
[115, 200, 191, 173, 172, 184, 205, 188, 180, 204, 169, 185, 186, 39] 7645
[115, 200, 191, 173, 172, 184, 205, 188, 180, 204, 169, 185, 

NetworkXNoPath: No path between 115 and 276.

In [17]:
p115to276 = nx.shortest_path(G, source = 115, target = 276, weight = "weight")

NetworkXNoPath: No path between 115 and 276.

In [21]:
p115to276 = nx.shortest_path(G, source = 115, target = 180, weight = "weight")
type(p115to276)

list

In [ ]:
shortest_path(G, start_point_id, end_point_id, edges, weight = "weight")

In [ ]:
path_s_e = nx.shortest_path(G, source=start_point_id, target=end_point_id, weight= weight)
length_s_e = nx.shortest_path_length(G, source=start_point_id, target=end_point_id, weight= weight)
    
short_path_edges = edges.loc[edges.to_from.isin(list(pairwise(path_s_e))) | edges.from_to.isin(list(pairwise(path_s_e)))]
short_path_edges['linewidth'] = short_path_edges['weights'].apply(lambda x: np.ceil(x * 0.01 / 2))
    

In [ ]:
e_coordinate = pygeos.points(coordinates_pairs.e_coordinates[i])
e_id = find_nearest_node(e_coordinate, nodes)

In [ ]:
s_coordinate = (4.978, 52.361)
s_id = find_nearest_node(s_coordinate, nodes)
s_id

In [ ]:
e_coordinate = nodes.geometry[106]
e_id = find_nearest_node(e_coordinate, nodes)
e_id

In [ ]:
e_coordinate = (4.881, 52.341)
e_id = find_nearest_node(e_coordinate, nodes)
e_id

In [ ]:
# s_id = 115
# e_id = 168
path_s_e, length_s_e,short_path_edges = shortest_path(G,s_id,e_id,edges,weight="weight")

In [ ]:
fig, ax = plt.subplots(1, 1,figsize=(12,10))

gpd.GeoDataFrame(edges.copy()).plot(ax=ax,zorder=0)
gpd.GeoDataFrame(short_path_edges.copy()).plot(ax=ax,color='black',zorder=1,linewidth=short_path_edges['linewidth'])
#gpd.GeoDataFrame(short_path_edges.copy()).plot(ax=ax,color='black',zorder=1)  # plot without weight

In [ ]:
gpd.GeoDataFrame(grid.copy()).plot(edgecolor='black')

In [ ]:
gpd.GeoDataFrame(pygeos.centroid(grid.geometry.values),columns=['geometry']).plot()

In [ ]:
gpd.GeoDataFrame(grid_center_coordinates).plot()

# itertools to create unique pairs